In [ ]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.phrases.utils import generate_phrase_hash
from src.wiktionary.lookup import get_wiktionary_urls
import json
from src.phrases.search import (
    get_phrases_by_collection,
    get_verbs_from_phrases,
    get_vocab_from_phrases,
)
from src.llm_tools.challenge_generation import test_tool_use
from src.story import get_story, get_all_stories

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
story = get_story("A Test Story")

In [25]:
from langcodes import Language


story.get_published_stories(target_language="fr-FR")

[PublishedStory(source_language=Language.make(language='en', territory='GB'), target_language=Language.make(language='fr', territory='FR'), gcs_path='stories/en-GB/fr-FR/a_test_story_232dea/', active=True, public_url='https://storage.googleapis.com/audio-language-trainer-stories/stories/en-GB/fr-FR/a_test_story_232dea/index.html', title='A Test Story', deck='Pack01', collection='Test')]

In [2]:
response = test_tool_use()

In [11]:
response.scenarios[2].find_out

[QandA(question='What single-origin beans do you have available today?', answer='Ethiopian Yirgacheffe, Colombian Supremo, and Kenyan AA'),
 QandA(question='What flavor notes does the Ethiopian coffee have?', answer='Bright citrus, floral jasmine, and blueberry undertones'),
 QandA(question='What brewing method would you recommend for the Colombian beans?', answer='Pour-over or French press to highlight its chocolate and nutty flavors'),
 QandA(question='How much does a pour-over cost?', answer='$6.50 for a single cup'),
 QandA(question='What is the roast date of the beans?', answer='Roasted three days ago on Monday')]

In [ ]:
COLLECTION = "LM1000"
DECK = "Pack01"

phrases = get_phrases_by_collection(COLLECTION, DECK)

In [ ]:
p = phrases[0]
get_wiktionary_urls(p.translations["sr-RS"].tokens, "sr")

In [ ]:
len(get_vocab_from_phrases(phrases))

In [ ]:
get_verbs_from_phrases(phrases)

In [ ]:
text = "Measure the pH level"

In [ ]:
tokens = text.split(" ")

In [ ]:
_get_wiktionary_url

In [ ]:
get_wiktionary_url("pH", "pH", "en")

In [ ]:
with open(
    "../outputs/wiktionary_dump/wiktionary_dump.json", "r", encoding="utf-8"
) as f:
    for step, line in enumerate(f):
        data = json.loads(line)
        print(data)
        break

In [ ]:
"Sam! Har du sett det här offentliga meddelandet om allemansrätten? Det öppnas en ny vandringsled imorgon."

In [ ]:
story_title = "A Test Story"
story_summary = "A brief summary of the test story."
story_title_hash = generate_phrase_hash(story_title)
story_dialogue = load_json("../data/test_story.json")

story_title2 = "A Test Story 2"
story_summary2 = "A brief summary of the test story 2."
story_title_hash2 = generate_phrase_hash(story_title2)
story_dialogue2 = load_json("../data/test_story2.json")

In [ ]:
ts1 = Story.create(
    story_title, story_summary, story_dialogue, collection="Test", deck="Pack01"
)
ts2 = Story.create(
    story_title2, story_summary2, story_dialogue2, collection="Test2", deck="Pack01"
)
ts1.upload(overwrite=True)
ts2.upload(overwrite=True)